# LANGGRAPH DESIGN PATTERNS - Complete Reference Guide

    This file demonstrates 6 core LangGraph patterns for building AI workflows:
    
    1. PROMPT CHAINING     - Sequential LLM calls with quality gates
    2. PARALLELIZATION     - Run multiple LLMs simultaneously
    3. ROUTING             - Dynamic routing based on input
    4. ORCHESTRATOR-WORKER - Divide work among parallel workers
    5. EVALUATOR-OPTIMIZER - Self-improving loops with feedback
    6. AGENTS              - LLMs that use tools
    
    PATTERN COMPARISON:
    ┌────────────────────┬──────────────────┬─────────────────────────────┐
    │ Pattern            │ When to Use      │ Example Use Case            │
    ├────────────────────┼──────────────────┼─────────────────────────────┤
    │ Prompt Chaining    │ Step-by-step     │ Draft → Edit → Polish       │
    │ Parallelization    │ Independent work │ Generate story + joke + poem│
    │ Routing            │ Conditional flow │ Route to story/joke/poem    │
    │ Orchestrator       │ Split work       │ Write report sections       │
    │ Evaluator          │ Quality control  │ Generate → Grade → Retry    │
    │ Agents             │ Use tools        │ Calculator with arithmetic  │
    └────────────────────┴──────────────────┴─────────────────────────────┘

## SETUP: Initialize LLM

In [1]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="qwen3:8b",
    model_provider="ollama",
    temperature=0  # 0 = deterministic (same input → same output)
)

C:\Users\ASUS\miniconda3\envs\Ollama\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## FOUNDATION: LLM Augmentations

    Before diving into patterns, understand how to AUGMENT LLMs:
    
    1. STRUCTURED OUTPUT - Force LLM to return specific format
    2. TOOL BINDING - Give LLM access to functions
    
    These augmentations are the building blocks for all patterns.

### Augmentation 1: Structured Output

    Schema for structured output.
    
    Pydantic models define EXACTLY what fields the LLM should return.
    
    Without this: LLM might return free-form text
    With this: LLM MUST return dict matching this schema
    
    Visual Comparison:
    ┌─────────────────────────────────────────────────────────────────────────┐
    │ Regular LLM:                                                            │
    │ Input: "How does X relate to Y?"                                        │
    │ Output: "Here's a search query: 'X and Y relationship'. This is         │
    │          relevant because..."  ← Unstructured text                      │
    └─────────────────────────────────────────────────────────────────────────┘
    
    ┌─────────────────────────────────────────────────────────────────────────┐
    │ Structured LLM:                                                         │
    │ Input: "How does X relate to Y?"                                        │
    │ Output: {                                                               │
    │   "search_query": "X and Y relationship",                               │
    │   "justification": "This query directly addresses..."                   │
    │ } ← Guaranteed dict format                                              │
    └─────────────────────────────────────────────────────────────────────────┘

In [2]:
from pydantic import BaseModel, Field

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query that is optimized web search.")
    justification: str = Field(
        None, description="Why this query is relevant to the user's request."
    )

# Augment the LLM with schema
structured_llm = llm.with_structured_output(SearchQuery)

# Invoke - guaranteed to return SearchQuery format
print("\n[Example: Structured Output]")
output = structured_llm.invoke("How does Calcium CT score relate to high cholesterol?")
print(f"Search Query: {output.search_query}")
print(f"Justification: {output.justification}")


[Example: Structured Output]
Search Query: How does Calcium CT score relate to high cholesterol?
Justification: The user is asking about the relationship between Calcium CT score and high cholesterol. This is a medical question that requires an explanation of how these two factors are connected in terms of cardiovascular health. The answer should cover the role of calcium in arteries, the impact of high cholesterol on plaque formation, and how the Calcium CT score is used to assess cardiovascular risk. It should also mention the importance of lifestyle changes and medical management for both conditions. The answer should be clear, concise, and provide actionable information for the user to understand the connection and take appropriate steps for their health. The user might be concerned about their own health or that of a family member, so the response should be informative and reassuring, emphasizing the importance of early detection and management. The answer should also highlight t

### Augmentation 2: Tool Binding

    Tool Binding Flow:
    ┌──────────────────────────────────────────────────────────────────────┐
    │ User: "What is 2 times 3?"                                           │
    └────────────────────┬─────────────────────────────────────────────────┘
                         │
                         ▼
              ┌──────────────────────┐
              │  llm_with_tools      │
              │  (knows about tools) │
              └──────────┬───────────┘
                         │
            ┌────────────┴───────────┐
            │ Option A: Call tool    │ Option B: Answer directly
            │ tool_calls=[           │ content="6"
            │   {name: "multiply",   │
            │    args: {a:2, b:3}}   │
            │ ]                      │
            └────────────────────────┘

In [3]:
# Define a tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

# Augment the LLM with tools
llm_with_tools = llm.bind_tools([multiply])

# Invoke with math question
print("\n[Example: Tool Binding]")
msg = llm_with_tools.invoke("What is 2 times 3?")

# Check if LLM made a tool call
if msg.tool_calls:
    print(f"LLM decided to use tool: {msg.tool_calls[0]['name']}")
    print(f"With arguments: {msg.tool_calls[0]['args']}")
else:
    print(f"LLM responded directly: {msg.content}")


[Example: Tool Binding]
LLM decided to use tool: multiply
With arguments: {'a': 2, 'b': 3}


## PATTERN 1: PROMPT CHAINING

    PROMPT CHAINING: Sequential LLM calls where each call builds on the previous.
    
    Use Case: Multi-step refinement (draft → improve → polish)
    
    Flow:
        Generate → Check Quality → Improve → Polish → Done
             │            │           │          │
             └────────────┴───────────┴──────────┘
                  Each step uses previous output
    
    Real-World Examples:
    - Content creation: Draft → SEO optimize → Fact-check → Publish
    - Code generation: Scaffold → Implement → Test → Document
    - Email writing: Draft → Tone adjustment → Grammar check → Send

In [4]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

### Define State

        State tracks the joke as it progresses through refinement steps.
        
        Visual State Evolution:
        Step 1: {topic: "cats", joke: "Why did cat cross road? To get to other side"}
        Step 2: {topic: "cats", joke: "...", improved_joke: "Why don't cats play poker? Too many cheetahs!"}
        Step 3: {topic: "cats", joke: "...", improved_joke: "...", final_joke: "Why don't cats play poker in the jungle? Too many cheetahs and the stakes are too high!"}

In [5]:
class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str

### Define Nodes (Each step in the chain)

    STEP 1: Generate initial joke.
    
    This is the first LLM call in the chain.

In [6]:
def generate_joke(state: State):
    msg = llm.invoke(f"Write a short joke about {state['topic']}")
    return {"joke": msg.content}

    QUALITY GATE: Check if joke has proper structure.
    
    This is a CONDITIONAL NODE - it decides which path to take.
    - If joke has punctuation (?, !) → Pass (good enough)
    - If joke lacks punctuation → Fail (needs improvement)
    
    Visual:
                ┌───────────────┐
                │check_punchline│
                └───────┬───────┘
                        │
            ┌───────────┴───────────┐
            │                       │
            ▼                       ▼
      ┌─────────┐             ┌─────────┐
      │  Pass   │             │  Fail   │
      │  → END  │             │  → Fix  │
      └─────────┘             └─────────┘

In [7]:
def check_punchline(state: State):
    # Simple check - does the joke contain "?" or "!"
    if "?" in state["joke"] or "!" in state["joke"]:
        return "Pass"
    return "Fail"

    STEP 2: Improve the joke (only if it failed the check).
    
    This call uses the OUTPUT of step 1 as INPUT.
    That's the "chaining" part!

In [8]:
def improve_joke(state: State):
    msg = llm.invoke(f"Make this joke funnier by adding wordplay: {state['joke']}")
    return {"improved_joke": msg.content}

    STEP 3: Final polish.
    
    This call uses the OUTPUT of step 2 as INPUT.

In [9]:
def polish_joke(state: State):
    msg = llm.invoke(f"Add a surprising twist to this joke: {state['improved_joke']}")
    return {"final_joke": msg.content}

### Build the Graph

In [10]:
workflow = StateGraph(State)

# Add nodes
workflow.add_node("generate_joke", generate_joke)
workflow.add_node("improve_joke", improve_joke)
workflow.add_node("polish_joke", polish_joke)

# Add edges
workflow.add_edge(START, "generate_joke")

# Conditional edge based on quality check
workflow.add_conditional_edges(
    "generate_joke",
    check_punchline,  # Function that returns "Pass" or "Fail"
    {
        "Fail": "improve_joke",  # If fails → improve it
        "Pass": END              # If passes → done!
    }
)

workflow.add_edge("improve_joke", "polish_joke")
workflow.add_edge("polish_joke", END)

# Compile
chain = workflow.compile()

print("\n[Graph Visualization]")
print("START → generate_joke → check → [Pass → END | Fail → improve → polish → END]")

# Run it!
print("\n[Running Prompt Chain]")
state = chain.invoke({"topic": "cats"})

print("\nInitial joke:")
print(state["joke"])

if "improved_joke" in state:
    print("\n--- Needed improvement ---")
    print("\nImproved joke:")
    print(state["improved_joke"])
    print("\nFinal joke:")
    print(state["final_joke"])
else:
    print("\n--- Passed quality check immediately ---")


[Graph Visualization]
START → generate_joke → check → [Pass → END | Fail → improve → polish → END]

[Running Prompt Chain]

Initial joke:
<think>
Okay, the user wants a short joke about cats. Let me think about common cat-related humor. Cats are known for their independence, knocking things over, and their love for boxes. Maybe play on those traits.

Hmm, maybe something about them being mischievous. Like how they knock things over. Wait, there's a classic joke about cats and the internet. Oh right, the "why did the cat..." format. Let me try that.

"Why did the cat bring a ladder to the party?" Maybe the punchline is about reaching the tuna? Wait, cats love tuna. But maybe something about climbing. Wait, the classic one is "Why did the cat fall off the roof? Because it was a cat and the roof was a cat." No, that's not right. Let me think again.

Alternatively, maybe a pun on "climbing." Like, "Why did the cat bring a ladder to the party? To reach the tuna." Wait, that's similar to th

## PATTERN 2: PARALLELIZATION

    PARALLELIZATION: Run multiple LLM calls simultaneously.
    
    Use Case: Independent tasks that don't depend on each other
    
    Flow:
                  START
                    │
            ┌───────┼───────┐
            │       │       │
            ▼       ▼       ▼
          LLM_1   LLM_2   LLM_3  ← All run at same time!
            │       │       │
            └───────┼───────┘
                    │
                    ▼
              Aggregator
                    │
                    ▼
                  END
    
    Performance:
    - Sequential: 3 LLM calls × 2 seconds each = 6 seconds total
    - Parallel:   3 LLM calls running simultaneously = 2 seconds total
    
    Real-World Examples:
    - Social media: Generate post + image caption + hashtags (parallel)
    - Research: Summarize multiple documents simultaneously
    - Content: Write blog post intro + body + conclusion in parallel

### Define State

In [11]:
class State(TypedDict):
    """State for parallel execution"""
    topic: str
    joke: str
    story: str
    poem: str
    combined_output: str

### Define Parallel Nodes

In [12]:
def call_llm_1(state: State):
    """Worker 1: Generate joke"""
    msg = llm.invoke(f"Write a joke about {state['topic']}")
    return {"joke": msg.content}


def call_llm_2(state: State):
    """Worker 2: Generate story"""
    msg = llm.invoke(f"Write a story about {state['topic']}")
    return {"story": msg.content}


def call_llm_3(state: State):
    """Worker 3: Generate poem"""
    msg = llm.invoke(f"Write a poem about {state['topic']}")
    return {"poem": msg.content}


def aggregator(state: State):
    """
    Combine results from all parallel workers.
    
    This node runs AFTER all parallel workers complete.
    LangGraph automatically waits for all parallel branches.
    """
    combined = f"Here's a story, joke, and poem about {state['topic']}!\n\n"
    combined += f"STORY:\n{state['story']}\n\n"
    combined += f"JOKE:\n{state['joke']}\n\n"
    combined += f"POEM:\n{state['poem']}"
    return {"combined_output": combined}

### Build Parallel Graph

In [14]:
parallel_builder = StateGraph(State)

# Add nodes
parallel_builder.add_node("call_llm_1", call_llm_1)
parallel_builder.add_node("call_llm_2", call_llm_2)
parallel_builder.add_node("call_llm_3", call_llm_3)
parallel_builder.add_node("aggregator", aggregator)

# CRITICAL: Multiple edges from START mean PARALLEL execution
parallel_builder.add_edge(START, "call_llm_1")
parallel_builder.add_edge(START, "call_llm_2")
parallel_builder.add_edge(START, "call_llm_3")

# All workers feed into aggregator
parallel_builder.add_edge("call_llm_1", "aggregator")
parallel_builder.add_edge("call_llm_2", "aggregator")
parallel_builder.add_edge("call_llm_3", "aggregator")

parallel_builder.add_edge("aggregator", END)

# Compile
parallel_workflow = parallel_builder.compile()

print("\n[Running Parallel Workflow]")
state = parallel_workflow.invoke({"topic": "cats"})
print(state["combined_output"])


[Running Parallel Workflow]
Here's a story, joke, and poem about cats!

STORY:
<think>
Okay, the user wants a story about cats. Let me think about how to approach this. First, I need to decide on the genre. Maybe a fantasy or magical realism story since cats often have a mystical side. Let me go with magical realism. 

I should create a setting. A small village sounds cozy. Maybe a village where cats have some special abilities. Wait, the user might want something unique. How about a village where cats are guardians of something? Maybe a hidden realm? 

Characters: A young cat protagonist. Let's name her Luna. She's curious and adventurous. Maybe she discovers her heritage. Her grandmother could be a wise figure, like a guardian. 

Conflict: Luna finds an ancient book that reveals her family's role as guardians. She needs to protect the village from a threat. Maybe a curse or a dark force. 

Plot points: Luna finds the book, learns about her destiny, faces challenges, and ultimately s

## PATTERN 3: ROUTING

    ROUTING: Dynamic path selection based on input.
    
    Use Case: Different inputs need different handling
    
    Flow:
             START
               │
               ▼
          ┌─────────┐
          │ Router  │ ← LLM decides which path
          └────┬────┘
               │
        ┌──────┼──────┐
        │      │      │
        ▼      ▼      ▼
      Story  Joke  Poem  ← Only ONE executes
        │      │      │
        └──────┼──────┘
               │
               ▼
             END
    
    Comparison with Parallelization:
    - Parallel: Run ALL branches
    - Routing: Run ONE branch (chosen dynamically)
    
    Real-World Examples:
    - Customer support: Route to FAQ / human / chatbot based on query
    - Content moderation: Route to auto-approve / review / reject
    - Email triage: Route to urgent / normal / spam based on content

In [15]:
from typing_extensions import Literal
from langchain.messages import HumanMessage, SystemMessage

### Define Routing Schema

    Structured output for routing decision.
    
    The LLM returns this to tell us which path to take.

In [16]:
class Route(BaseModel):
    step: Literal["poem", "story", "joke"] = Field(
        None, description="The next step in the routing process"
    )

# Create router (LLM that returns Route)
router = llm.with_structured_output(Route)

### Define State

In [17]:
class State(TypedDict):
    input: str      # User's request
    decision: str   # Router's decision
    output: str     # Final output

### Define Nodes (One for each path)

In [18]:
def llm_call_1(state: State):
    """Path 1: Write a story"""
    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_2(state: State):
    """Path 2: Write a joke"""
    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_3(state: State):
    """Path 3: Write a poem"""
    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_router(state: State):
    """
    Router node: Decides which path to take.
    
    Uses structured output to make routing decision.
    """
    decision = router.invoke(
        [
            SystemMessage(
                content="Route the input to story, joke, or poem based on the user's request."
            ),
            HumanMessage(content=state["input"]),
        ]
    )
    return {"decision": decision.step}

### Define Routing Logic

    Conditional edge function: Maps decision to node name.
    
    This function returns the NAME of the next node to visit.
    
    Flow:
        decision="story" → return "llm_call_1"
        decision="joke"  → return "llm_call_2"
        decision="poem"  → return "llm_call_3"

In [19]:
def route_decision(state: State):
    if state["decision"] == "story":
        return "llm_call_1"
    elif state["decision"] == "joke":
        return "llm_call_2"
    elif state["decision"] == "poem":
        return "llm_call_3"

### Build Routing Graph

In [20]:
router_builder = StateGraph(State)

# Add nodes
router_builder.add_node("llm_call_1", llm_call_1)
router_builder.add_node("llm_call_2", llm_call_2)
router_builder.add_node("llm_call_3", llm_call_3)
router_builder.add_node("llm_call_router", llm_call_router)

# Add edges
router_builder.add_edge(START, "llm_call_router")

# Conditional routing
router_builder.add_conditional_edges(
    "llm_call_router",
    route_decision,  # Function that returns node name
    {
        # Map of: return value → node name
        "llm_call_1": "llm_call_1",
        "llm_call_2": "llm_call_2",
        "llm_call_3": "llm_call_3",
    },
)

# All paths lead to END
router_builder.add_edge("llm_call_1", END)
router_builder.add_edge("llm_call_2", END)
router_builder.add_edge("llm_call_3", END)

# Compile
router_workflow = router_builder.compile()

print("\n[Running Routing Workflow]")
state = router_workflow.invoke({"input": "Write me a joke about cats"})
print(f"Router decision: {state.get('decision', 'N/A')}")
print(f"Output: {state['output'][:100]}...")


[Running Routing Workflow]
Router decision: joke
Output: <think>
Okay, the user wants a joke about cats. Let me think about common cat-related humor. Cats ar...


## PATTERN 4: ORCHESTRATOR-WORKER

    ORCHESTRATOR-WORKER: One LLM plans, many LLMs execute.
    
    Use Case: Divide large task into subtasks, execute in parallel
    
    Flow:
               START
                 │
                 ▼
          ┌──────────────┐
          │ Orchestrator │ ← Creates plan: [task1, task2, task3]
          └──────┬───────┘
                 │
          ┌──────┼──────┐
          │      │      │
          ▼      ▼      ▼
       Worker Worker Worker  ← Each handles one task (parallel)
          │      │      │
          └──────┼──────┘
                 │
                 ▼
          ┌──────────────┐
          │ Synthesizer  │ ← Combines results
          └──────────────┘
                 │
                 ▼
               END
    
    Real-World Examples:
    - Report writing: Plan sections → Write each section → Combine
    - Research: Break query into sub-questions → Answer each → Synthesize
    - Code generation: Plan modules → Generate each → Integrate

In [21]:
from typing import Annotated, List
import operator

### Define Planning Schema

In [22]:
class Section(BaseModel):
    """Schema for one section of a report"""
    name: str = Field(description="Name for this section of the report.")
    description: str = Field(
        description="Brief overview of the main topics and concepts to be covered in this section."
    )


class Sections(BaseModel):
    """Schema for the complete plan"""
    sections: List[Section] = Field(description="Sections of the report.")

# Create planner (LLM that returns Sections)
planner = llm.with_structured_output(Sections)

### Define States

    Main graph state.
    
    The Annotated[list, operator.add] is CRITICAL:
    - Multiple workers write to completed_sections simultaneously
    - operator.add means: APPEND to list (don't replace)
    - Without this, workers would overwrite each other!

In [23]:
class State(TypedDict):
    topic: str
    sections: list[Section]
    completed_sections: Annotated[list, operator.add]  # Workers append here
    final_report: str

    Individual worker state.
    
    Each worker gets ONE section to work on.

In [24]:
class WorkerState(TypedDict):
    section: Section
    completed_sections: Annotated[list, operator.add]

### Define Nodes

    ORCHESTRATOR: Creates the plan.
    
    This node:
    1. Takes the topic
    2. Breaks it into sections
    3. Returns list of sections for workers

In [25]:
def orchestrator(state: State):
    report_sections = planner.invoke(
        [
            SystemMessage(content="Generate a plan for the report."),
            HumanMessage(content=f"Here is the report topic: {state['topic']}"),
        ]
    )
    return {"sections": report_sections.sections}

    WORKER: Writes one section.
    
    Multiple instances of this node run in parallel.
    Each worker gets a different section to write.
    
    Visual:
        Worker 1: section="Introduction"  → writes intro
        Worker 2: section="Methods"       → writes methods
        Worker 3: section="Results"       → writes results
                       (all at same time!)

In [26]:
def llm_call(state: WorkerState):
    section = llm.invoke(
        [
            SystemMessage(
                content="Write a report section following the provided name and description. Include no preamble for each section. Use markdown formatting."
            ),
            HumanMessage(
                content=f"Here is the section name: {state['section'].name} and description: {state['section'].description}"
            ),
        ]
    )
    # Write to completed_sections (operator.add appends)
    return {"completed_sections": [section.content]}

    SYNTHESIZER: Combines all sections.
    
    This node runs AFTER all workers complete.
    It takes all completed sections and combines them into final report.

In [27]:
def synthesizer(state: State):
    completed_sections = state["completed_sections"]
    completed_report_sections = "\n\n---\n\n".join(completed_sections)
    return {"final_report": completed_report_sections}

### Define Worker Assignment Logic

    Create one worker for each section.
    
    The Send() API is CRITICAL for orchestrator-worker pattern:
    - Send("node_name", {"data": ...}) creates a parallel execution
    - Each Send creates an independent worker
    - All workers run simultaneously
    
    Visual:
        sections = [Section1, Section2, Section3]
        
        Returns: [
            Send("llm_call", {"section": Section1}),
            Send("llm_call", {"section": Section2}),
            Send("llm_call", {"section": Section3})
        ]
        
        Result: 3 parallel workers, each writing one section

In [28]:
from langgraph.types import Send

def assign_workers(state: State):
    return [Send("llm_call", {"section": s}) for s in state["sections"]]

### Build Orchestrator-Worker Graph

In [29]:
orchestrator_worker_builder = StateGraph(State)

# Add nodes
orchestrator_worker_builder.add_node("orchestrator", orchestrator)
orchestrator_worker_builder.add_node("llm_call", llm_call)
orchestrator_worker_builder.add_node("synthesizer", synthesizer)

# Add edges
orchestrator_worker_builder.add_edge(START, "orchestrator")

# CRITICAL: assign_workers creates dynamic parallel workers
orchestrator_worker_builder.add_conditional_edges(
    "orchestrator",
    assign_workers,  # Returns list of Send() objects
    ["llm_call"]     # Worker node name
)

orchestrator_worker_builder.add_edge("llm_call", "synthesizer")
orchestrator_worker_builder.add_edge("synthesizer", END)

# Compile
orchestrator_worker = orchestrator_worker_builder.compile()

print("\n[Running Orchestrator-Worker Pattern]")
state = orchestrator_worker.invoke({"topic": "Create a report on LLM scaling laws"})
print(f"Generated report with {len(state['sections'])} sections")
print(f"Final report length: {len(state['final_report'])} characters")


[Running Orchestrator-Worker Pattern]
Generated report with 8 sections
Final report length: 32820 characters


In [31]:
print(state['final_report'])

<think>
Okay, the user wants me to write a report section called "Introduction and description" about LLM scaling laws, their significance in AI, and their impact on developing large language models. Let me start by recalling what LLM scaling laws are. They refer to the observation that as the size of a language model increases, its performance on various tasks improves, often following a power-law relationship. 

First, I need to define scaling laws clearly. Maybe mention the key studies like the one by Kaplan et al. from 2020. They showed that model performance scales with parameters and data. Then, explain the significance: why this matters for AI. It's about understanding how to optimize resources and predict performance gains. 

Next, the impact on large language models. I should talk about how scaling laws guide the development of larger models, leading to better capabilities. Also, mention the trade-offs, like computational costs and energy use. Maybe touch on the shift from sma

## PATTERN 5: EVALUATOR-OPTIMIZER

    EVALUATOR-OPTIMIZER: Self-improving loop with quality feedback.
    
    Use Case: Generate content, evaluate quality, retry if needed
    
    Flow:
             START
               │
               ▼
          ┌──────────┐
          │Generator │ ← Creates content
          └─────┬────┘
                │
                ▼
          ┌──────────┐
          │Evaluator │ ← Grades quality
          └─────┬────┘
                │
          ┌─────┴─────┐
          │           │
          ▼           ▼
       Accept      Reject
        END      (with feedback)
                  │
                  └──► Loop back to Generator
    
    This creates a LOOP until quality threshold is met.
    
    Real-World Examples:
    - Code generation: Generate → Test → Fix → Repeat until tests pass
    - Content creation: Draft → Grade → Revise → Repeat until acceptable
    - Translation: Translate → Evaluate accuracy → Retry → Repeat
    - Image generation: Generate → Check quality → Regenerate with feedback
    
    EXECUTION TRACE:
    ─────────────────────────────────────────────────────────────────────────
    Iteration 1:
      Generator: "Why did the cat cross the road?"
      Evaluator: Grade = "not funny", Feedback = "Too generic, add wordplay"
      Decision: REJECT → Loop back
    
    Iteration 2:
      Generator (with feedback): "Why don't cats play poker? Too many cheetahs!"
      Evaluator: Grade = "funny", Feedback = ""
      Decision: ACCEPT → END
    ─────────────────────────────────────────────────────────────────────────

In [32]:
from langgraph.graph import StateGraph, START, END

### Define State

    State tracks content through improvement loop.
    
    Fields:
        topic: What to generate content about
        joke: Current version of the joke
        feedback: Feedback from evaluator (if rejected)
        funny_or_not: Evaluation result ("funny" or "not funny")

In [33]:
class State(TypedDict):
    joke: str
    topic: str
    feedback: str
    funny_or_not: str

### Define Evaluation Schema

    Structured output for evaluation.
    
    The evaluator LLM returns this to tell us:
    - Is the content acceptable?
    - If not, how to improve it?
    
    Example Output:
    {
        "grade": "not funny",
        "feedback": "The punchline is too predictable. Try adding unexpected wordplay."
    }

In [34]:
class Feedback(BaseModel):
    grade: Literal["funny", "not funny"] = Field(
        description="Decide if the joke is funny or not.",
    )
    feedback: str = Field(
        description="If the joke is not funny, provide feedback on how to improve it.",
    )

# Create evaluator (LLM that returns Feedback)
evaluator = llm.with_structured_output(Feedback)

### Define Nodes

    GENERATOR: Creates content (with optional feedback incorporation).
    
    Two modes:
    1. First attempt: Generate from scratch
    2. Retry: Incorporate feedback from evaluator
    
    Visual Loop:
    ┌──────────────────────────────────────────────────────────────┐
    │ Iteration 1:                                                 │
    │   No feedback → Generate: "Why did cat cross road?"          │
    │                                                              │
    │ Evaluator: "Not funny - too generic"                         │
    │                                                              │
    │ Iteration 2:                                                 │
    │   With feedback → Generate: "Why don't cats play poker?      │
    │                             Too many cheetahs!"              │
    │                                                              │
    │ Evaluator: "Funny!" → Accept                                 │
    └──────────────────────────────────────────────────────────────┘
    
    Args:
        state: Contains topic and optional feedback
        
    Returns:
        Updated state with new joke

In [35]:
def llm_call_generator(state: State):
    if state.get("feedback"):
        # RETRY MODE: Incorporate feedback
        msg = llm.invoke(
            f"Write a joke about {state['topic']} but take into account the feedback: {state['feedback']}"
        )
    else:
        # FIRST ATTEMPT: Generate from scratch
        msg = llm.invoke(f"Write a joke about {state['topic']}")
    
    return {"joke": msg.content}

    EVALUATOR: Grades the content quality.
    
    This node decides if we're done or need to retry.
    
    Process:
    1. Take the current joke
    2. Ask LLM to evaluate it
    3. Get structured response (grade + feedback)
    4. Return evaluation results
    
    Returns:
        - funny_or_not: "funny" or "not funny"
        - feedback: How to improve (if not funny)

In [36]:
def llm_call_evaluator(state: State):
    grade = evaluator.invoke(f"Grade the joke {state['joke']}")
    return {
        "funny_or_not": grade.grade,
        "feedback": grade.feedback
    }

### Define Routing Logic

    Route based on evaluation: Accept or Reject with feedback.
    
    Decision Tree:
                    ┌───────────┐
                    │Evaluator  │
                    └─────┬─────┘
                          │
              ┌───────────┴──────────┐
              │                      │
              ▼                      ▼
        ┌──────────┐          ┌─────────────┐
        │  Funny   │          │ Not Funny   │
        │  → END   │          │ → Retry     │
        └──────────┘          └─────────────┘
                                    │
                                    └──► Loop back to Generator
    
    Returns:
        "Accepted": Go to END (we're done!)
        "Rejected + Feedback": Go back to generator (try again)

In [37]:
def route_joke(state: State):
    if state["funny_or_not"] == "funny":
        return "Accepted"
    elif state["funny_or_not"] == "not funny":
        return "Rejected + Feedback"

### Build Evaluator-Optimizer Graph

In [38]:
optimizer_builder = StateGraph(State)

# Add nodes
optimizer_builder.add_node("llm_call_generator", llm_call_generator)
optimizer_builder.add_node("llm_call_evaluator", llm_call_evaluator)

# Add edges
optimizer_builder.add_edge(START, "llm_call_generator")
optimizer_builder.add_edge("llm_call_generator", "llm_call_evaluator")

# Conditional edge creates the LOOP
optimizer_builder.add_conditional_edges(
    "llm_call_evaluator",
    route_joke,
    {
        "Accepted": END,                             # Good enough → stop
        "Rejected + Feedback": "llm_call_generator"  # Not good → retry
    },
)

# Compile
optimizer_workflow = optimizer_builder.compile()

print("\n[Running Evaluator-Optimizer Pattern]")
state = optimizer_workflow.invoke({"topic": "Cats"})
print(f"Final joke: {state['joke']}")
print(f"Quality: {state['funny_or_not']}")


[Running Evaluator-Optimizer Pattern]
Final joke: <think>
Okay, the user wants a joke about cats. Let me think about common cat-related humor. Cats are known for their independence, knocking things over, and their love for boxes. Maybe play on those traits.

Hmm, what's a typical situation where a cat causes a funny problem? Maybe something with their curiosity leading to a mishap. Like knocking over something important. Maybe a keyboard? Because that's relatable.

Wait, the joke should have a setup and a punchline. Let me try: "Why did the cat get stuck in the microwave?" Maybe the punchline is something about being a "purr-fect fit." Wait, that's a play on "perfect fit" and "pur." But maybe that's too forced. Let me think again.

Alternatively, maybe a pun on "meow." Like, "Why don't cats ever get cold? Because they always have nine lives and a sweater!" No, that's not very funny. Maybe something about their behavior. 

How about this: "Why did the cat bring a ladder to the party? B

## PATTERN 6: AGENTS (Tool-Using LLMs)

    AGENTS: LLMs that decide when and how to use tools.
    
    Use Case: LLMs need to perform actions (calculations, API calls, etc.)
    
    Flow:
             START
               │
               ▼
          ┌─────────┐
          │   LLM   │ ← "I need to calculate 3+4"
          └────┬────┘
               │
          ┌────┴────┐
          │         │
          ▼         ▼
      Use Tool   Respond
          │       Directly
          │         │
          ▼         │
      ┌───────┐     │
      │Execute│     │
      │Tool   │     │
      └───┬───┘     │
          │         │
          └────┬────┘
               │
               ▼
          ┌─────────┐
          │   LLM   │ ← "The answer is 7"
          └────┬────┘
               │
               ▼
              END
    
    This is the AGENTIC LOOP: Think → Act → Observe → Think → ...
    
    WHY AGENTS MATTER:
    ──────────────────────────────────────────────────────────────────
    Problem: LLMs are bad at math
      User: "What is 3 + 4?"
      LLM: "3 + 4 = 7.2"  ← Sometimes wrong!
    
    Solution: Give LLM tools
      User: "What is 3 + 4?"
      LLM: "I'll use the add tool"
      Tool: add(3, 4) = 7  ← Always correct!
      LLM: "The answer is 7"
    ──────────────────────────────────────────────────────────────────
    
    Real-World Examples:
    - Calculator agent: Math questions → Use calculator tools
    - Research agent: Questions → Search web → Synthesize
    - Database agent: Queries → SQL tools → Format results
    - API agent: Tasks → Call APIs → Process responses

In [39]:
from langchain.tools import tool
from langgraph.graph import MessagesState
from langchain.messages import SystemMessage, HumanMessage, ToolMessage

### Define Tools

    Tools are functions the LLM can call.
    
    The @tool decorator:
    1. Extracts function signature
    2. Reads docstring (LLM uses this to understand the tool!)
    3. Creates a schema the LLM can understand
    
    CRITICAL: Good docstrings = LLM knows when to use the tool

In [40]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply `a` and `b`.
    
    Use this when the user asks to multiply numbers.
    
    Args:
        a: First integer to multiply
        b: Second integer to multiply
        
    Returns:
        The product of a and b
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Adds `a` and `b`.
    
    Use this when the user asks to add or sum numbers.
    
    Args:
        a: First integer to add
        b: Second integer to add
        
    Returns:
        The sum of a and b
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """Divide `a` by `b`.
    
    Use this when the user asks to divide numbers.
    
    Args:
        a: Numerator (number to be divided)
        b: Denominator (number to divide by)
        
    Returns:
        The quotient of a divided by b
    """
    return a / b

### Bind Tools to LLM

    After binding, the LLM knows about these tools and can:
    1. Decide when to use them
    2. Choose the right tool for the task
    3. Provide arguments in the correct format
    
    Visual:
        Regular LLM:
            Input: "What is 3 + 4?"
            Output: "3 + 4 equals 7" (just text)
        
        LLM with tools:
            Input: "What is 3 + 4?"
            Output: AIMessage(
                tool_calls=[{
                    "name": "add",
                    "args": {"a": 3, "b": 4},
                    "id": "call_123"
                }]
            )

In [41]:
# Create list of tools
tools = [add, multiply, divide]

# Create lookup dictionary (for executing tools later)
tools_by_name = {tool.name: tool for tool in tools}
# Result: {"add": <add_tool>, "multiply": <multiply_tool>, "divide": <divide_tool>}

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

### Define Nodes

    LLM NODE: The "brain" that decides what to do.
    
    This node:
    1. Looks at conversation history
    2. Decides whether to:
       - Call a tool (e.g., "I need to use add(3, 4)")
       - Respond directly (e.g., "The answer is 7")
    
    Returns AIMessage with either:
    - tool_calls: List of tools to call (if needs tools)
    - content: Direct text response (if has answer)
    
    Example Flow:
    ────────────────────────────────────────────────────────────
    Messages: [HumanMessage("What is 3 + 4?")]
    ↓
    LLM thinks: "I should use the add tool for this"
    ↓
    Returns: AIMessage(tool_calls=[{name:"add", args:{a:3,b:4}}])
    ────────────────────────────────────────────────────────────

In [42]:
def llm_call(state: MessagesState):
    return {
        "messages": [
            llm_with_tools.invoke(
                [
                    SystemMessage(
                        content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
                    )
                ]
                + state["messages"]  # Add conversation history
            )
        ]
    }

    TOOL NODE: The "hands" that execute actions.
    
    This node:
    1. Extracts tool calls from last AI message
    2. For each tool call:
       - Look up tool by name
       - Execute with provided arguments
       - Create ToolMessage with result
    3. Return ToolMessages to append to conversation
    
    Detailed Process:
    ────────────────────────────────────────────────────────────────────
    Input: Last message has tool_calls=[{name:"add", args:{a:3,b:4}, id:"123"}]
    
    Step 1: Extract tool call
        tool_call = {name: "add", args: {a:3, b:4}, id: "123"}
    
    Step 2: Look up tool
        tool = tools_by_name["add"]  → <add function>
    
    Step 3: Execute tool
        result = add.invoke({a: 3, b: 4})  → 7
    
    Step 4: Create result message
        ToolMessage(content="7", tool_call_id="123")
    
    Output: {messages: [ToolMessage("7")]}
    ────────────────────────────────────────────────────────────────────
    
    The tool_call_id links the result back to the original request!
    This is how the LLM knows which tool call this result belongs to.

In [43]:
def tool_node(state: dict):
    result = []
    
    # Process each tool call
    for tool_call in state["messages"][-1].tool_calls:
        # Look up tool by name
        tool = tools_by_name[tool_call["name"]]
        
        # Execute the tool
        observation = tool.invoke(tool_call["args"])
        
        # Create ToolMessage with result
        result.append(
            ToolMessage(
                content=str(observation),      # The actual result
                tool_call_id=tool_call["id"]   # Links to original request
            )
        )
    
    return {"messages": result}

### Define Routing Logic

    Decide: Execute tool OR finish?
    
    This is the AGENTIC DECISION:
    - If LLM wants to use tools → Execute them
    - If LLM has final answer → We're done
    
    Decision Tree:
    ────────────────────────────────────────────────────────────
                      ┌──────────────┐
                      │Last message  │
                      └──────┬───────┘
                             │
                ┌────────────┴────────────┐
                │                         │
                ▼                         ▼
        ┌───────────────┐         ┌──────────────┐
        │Has tool_calls?│         │No tool_calls?│
        │     YES       │         │      NO      │
        └───────┬───────┘         └──────┬───────┘
                │                        │
                ▼                        ▼
        Go to tool_node              Go to END
        (execute tools)          (done - have answer)
    ────────────────────────────────────────────────────────────
    
    Args:
        state: Contains message history
        
    Returns:
        "tool_node": If LLM wants to use tools
        END: If LLM has final answer

In [44]:
def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    messages = state["messages"]
    last_message = messages[-1]
    
    # Check if LLM made a tool call
    if last_message.tool_calls:
        return "tool_node"  # Execute the tools
    
    return END  # LLM provided final answer

### Build Agent Graph

    Agent Graph Structure:
    ────────────────────────────────────────────────────────────────
             START
               │
               ▼
          ┌─────────┐
          │llm_call │ ◄─────────┐
          └────┬────┘           │
               │                │
        [should_continue?]      │
               │                │
          ┌────┴────┐           │
          ▼         ▼           │
      tool_node    END          │
          │                     │
          └─────────────────────┘
    
    The LOOP is critical:
    - tool_node → llm_call allows multiple tool calls
    - LLM can use one tool, see result, then use another tool
    - Continues until LLM has final answer
    ────────────────────────────────────────────────────────────────

In [45]:
agent_builder = StateGraph(MessagesState)

# Add nodes
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tool_node", tool_node)

# Add edges
agent_builder.add_edge(START, "llm_call")

# Conditional edge: tool_node or END?
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    ["tool_node", END]
)

# CRITICAL LOOP: After tool execution, go back to LLM
# This allows the agent to:
# 1. See the tool result
# 2. Decide if more tools are needed
# 3. Format the final answer
agent_builder.add_edge("tool_node", "llm_call")

# Compile
agent = agent_builder.compile()

print("\n[Running Agent Pattern]")
messages = [HumanMessage(content="Add 3 and 4.")]
result = agent.invoke({"messages": messages})

print("\nConversation trace:")
print("=" * 60)
for i, m in enumerate(result["messages"], 1):
    print(f"\n--- Message {i} ---")
    m.pretty_print()


[Running Agent Pattern]

Conversation trace:

--- Message 1 ---
================================ Human Message =================================

Add 3 and 4.

--- Message 2 ---
================================== Ai Message ==================================

<think>
Okay, the user wants to add 3 and 4. Let me check the available functions. There's the 'add' function which takes two integers, a and b. Since the user is asking to add 3 and 4, I should call the add function with a=3 and b=4. That should return the sum, which is 7. I need to make sure I use the correct parameters and format the tool call properly.
</think>
Tool Calls:
  add (54b5fb3a-652e-4fe8-9a11-71d16bde775c)
 Call ID: 54b5fb3a-652e-4fe8-9a11-71d16bde775c
  Args:
    a: 3
    b: 4

--- Message 3 ---
================================= Tool Message =================================

7

--- Message 4 ---
================================== Ai Message ==================================

<think>
Okay, the user asked to add 3